__BIOBSS - Heart Rate Variability (HRV) Analysis__

_This notebook includes guidelines to help using BIOBSS for calculating heart rate variability parameters from PPG or ECG signal._

In [1]:
#Import BIOBSS and the other required packages

import biobss
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Table of Contents
1. [Heart rate variability analysis using PPG signal](#hrv_ppg)<br>
    1.1. [PPG Sample Data](#ppg_data)<br>
    1.2. [PPG Preprocessing](#ppg_pre)<br>
    1.3. [Calculation of HRV Parameters](#ppg_par)<br>
2. [Heart rate variability analysis using ECG signal](#hrv_ecg)<br> 
    2.1. [ECG Sample Data](#ecg_data)<br>
    2.2. [ECG Preprocessing](#ecg_pre)<br>
    2.3. [Calculation of HRV Parameters](#ecg_par)<br>

Heart rate variability (HRV) analysis can be performed using either ECG or PPG signal. For this analysis, R peaks of ECG signal or systolic peaks of PPG signal should be detected first. Using peak locations, first peak to peak intervals (ppi) are derived and then HRV parameters are calculated.

HRV parameters can be grouped into three categories as time-domain parameters, frequency-domain parameters and nonlinear parameters. BIOBSS library provides 'get_hrv_features' function to calculate HRV parameters from ECG or PPG signal. Parameters for each domain can be calculated by defining feature_types parameter or using the functions specific to each category. All functions return a dictionary of the calculated features.


The HRV parameters which can be calculated using BIOBSS library are given below.

<b>Time-domain HRV parameters</b>:
- 'mean_nni': mean of peak to peak intervals
- 'sdnn': standard deviation of peak to peak intervals
- 'rmssd': root mean square of successive differences between peak to peak intervals
- 'sdsd': standard deviation of successive differences between peak to peak intervals
- 'nni_50': number of pairs of successive intervals that differ by more than 50 ms
- 'pnni_50': ratio of nni_50 to total number of intervals
- 'nni_20': number of pairs of successive intervals that differ by more than 20 ms
- 'pnni_20': ratio of nni_20 to total number of intervals
- 'cvnni': ratio of sdnn to mean_nni 
- 'cvsd': ratio of rmssd to mean_nni
- 'median_nni': median of absolute values of successive differences between peak to peak intervals
- 'range_nni': range of peak to peak intervals
- 'mean_hr': mean heart rate
- 'min_hr': minimum heart rate
- 'max_hr': maximum heart rate
- 'std_hr': standard deviation of heart rate
- 'mad_nni': mean absolute deviation of peak to peak intervals
- 'mcv_nni': ratio of mead_nni to median_nni
- 'iqr_nni': interquartile range of peak to peak intervals

<b>Frequency-domain HRV parameters</b>:
- 'vlf': spectral power pertaining to very low frequency band (0.0033 to 0.04 Hz by default.)
- 'lf': spectral power pertaining to low frequency band (0.04 to 0.15 Hz by default.)
- 'hf': spectral power pertaining to high frequency band (0.15 to 0.4 Hz by default.)
- 'lf_hf_ratio': ratio of lf to hf
- 'total_power': sum of vlf, lf and hf
- 'lfnu': normalized spectral power pertaining to low frequency band (ratio of lf to total_power)
- 'hfnu': normalized spectral power pertaining to high frequency band (ratio of hf to total_power)
- 'lnLF': log transformed low-frequency power
- 'lnHF': log transformed high-frequency power
- 'vlf_peak': max peak of power spectral density in very low frequency band
- 'lf_peak': max peak of power spectral density in low frequency band
- 'hf_peak': max peak of power spectral density in high frequency band

<b>Nonlinear HRV parameters</b>:
- 'SD1': standard deviation of Poincare plot perpendicular to the line of identity
- 'SD2': standard deviation of Poincare plot along the line of identity
- 'SD2_SD1': ratio of SD2 to SD1
- 'CSI': cardiac stress index
- 'CVI': cardiac vagal index
- 'CSI_mofidied': modified cardiac stress index
- 'ApEn': approximate entropy of peak to peak intervals
- 'SampEn': sample entropy of peak to peak intervals

### __Heart rate variability analysis using PPG signal__
<a id="hrv_ppg"></a>

#### __PPG Sample Data__
<a id="ppg_data"></a>

PPG sample data is provided as a csv file in BIOBSS\sample data. The data file contains 100 PPG segments of 10-seconds length. The sampling rate is 64 Hz for all segments.

In [ ]:
#Load the sample data
data_dir = os.path.join(os.path.normpath(os.getcwd() + os.sep + os.pardir),"sample_data")
filename='ppg_sample_data.csv'
data=pd.read_csv(os.path.join(data_dir,filename), header=None)

#Select the first segment to be used in the example
fs=64
L=10
sig=np.asarray(data.iloc[0,:])  

#### __PPG Preprocessing__
<a id="ppg_pre"></a>

In order to detect peaks accurately, PPG signal should be filtered first. BIOBSS provides a filtering function which uses Scipy. This function can be used to implement Butterworth filter by defining the filter parameters (filter type, filter order, cutoff frequencies). 

As an alternative, predefined filters can be used for each signal type. In order to use this option for PPG signal, signal_type should be selected as 'PPG'. 

In [ ]:
#Filter PPG signal by using predefined filters
filtered_ppg = biobss.preprocess.filter_signal(sig, sampling_rate=fs, signal_type='PPG', method='bandpass')

Next, PPG peaks should be detected to calculate interbeat (peak to peak) intervals (ibi, ppi). 

BIOBSS provides a peak detection function with different alternatives for the peak detection method. These methods are appropriate for PPG signal, however the parameters should be selected properly if the second peak (diastolic peak) is observable in the signal. The ___peak_detection___ function returns a dictionary including peak locations, peak amplitudes, trough locations and trough amplitudes.

In [ ]:
#Detect peaks using 'peakdet' method (delta=0.01). Delta parameter should be adjusted related to the amplitude of the signal.

info=biobss.preprocess.peak_detection(sig,fs,'peakdet',delta=0.01)

locs_peaks=info['Peak_locs']
peaks=info['Peaks']
locs_onsets=info['Trough_locs']
onsets=info['Troughs']

For further analysis, a peak control step is required to prevent errors resulting from incorrect peak detection results (missing or dupliciate peaks). The ___peak_control___ function checks the relative locations of peaks and troughs, ensuring only a single peak is located between consecutive troughs.

In [ ]:
#Correct the peak detection results by considering the order of peaks and troughs
info=biobss.ppgtools.peak_control(locs_peaks,peaks,locs_onsets,onsets)

locs_peaks=info['Peak_locs']
peaks=info['Peaks']
locs_onsets=info['Trough_locs']
onsets=info['Troughs']

#### __Calculation of HRV Parameters__
<a id="ppg_par"></a>

BIOBSS provides ___get_hrv_features___ function to calculate HRV parameters from PPG signal. This function can take 'ppi', 'peaks' or 'troughs' as argument for input type. If 'ppi' is selected as input_type, ppi intervals must be provided in milliseconds. If 'peaks' or 'troughs' is selected as input_type, ppi is calculated by the function itself using peak/trough locations and sampling rate.

In [ ]:
#Calculate HRV parameters using ppi intervals
ppi = 1000*np.diff(locs_peaks)/fs 
ppg_hrv = biobss.hrvtools.get_hrv_features(sampling_rate=fs, input_type='ppi',ppi=ppi)

In [ ]:
#Calculate HRV parameters using peak locations
ppg_hrv = biobss.hrvtools.get_hrv_features(sampling_rate=fs,input_type='peaks',peaks_locs=locs_peaks)

In [ ]:
#Calculate HRV parameters using trough locations
ppg_hrv = biobss.hrvtools.get_hrv_features(sampling_rate=fs,input_type='troughs',troughs_locs=locs_onsets)

The functions ___hrv_time_features___, ___hrv_freq_features___, ___hrv_nl_features___ can be used seperately in order to calculate hrv parameters for a specific domain. Note that, in order to use these functions peak to peak intervals should be provided as input in milliseconds.

In [ ]:
#Calculate time-domain HRV parameters 
ppg_hrv_time = biobss.hrvtools.hrv_time_features(ppi=ppi, sampling_rate=fs)

In [ ]:
#Calculate frequency-domain HRV parameters 
ppg_hrv_freq = biobss.hrvtools.hrv_freq_features(ppi=ppi, sampling_rate=fs)

In [ ]:
#Calculate nonlinear HRV parameters 
ppg_hrv_nl = biobss.hrvtools.hrv_nl_features(ppi=ppi, sampling_rate=fs)

### __Heart rate variability analysis using ECG signal__
<a id="hrv_ecg"></a>

#### __ECG Sample Data__
<a id="ecg_data"></a>

ECG sample data is provided as a csv file in BIOBSS\sample data. The data file contains an ECG signal of 5 minutes length, sampled at 256 Hz.

In [ ]:
#Load the sample data.

data_dir = os.path.join(os.path.normpath(os.getcwd() + os.sep + os.pardir),"sample_data")
filename='ecg_sample_data.csv'

data=pd.read_csv(os.path.join(data_dir,filename))
#Select the first segment to be used in the examples
fs=256
L=10
sig=np.asarray(data.iloc[:fs*L,0]) 

#### __ECG Preprocessing__
<a id="ecg_pre"></a>

In order to detect R peaks accurately, ECG signal should be filtered first. BIOBSS provides a filtering function which uses Scipy. This function can be used to implement Butterworth filter by defining the filter parameters (filter type, filter order, cutoff frequencies) as shown below.

As an alternative, predefined filters can be used for each signal type. In order to use this option for ECG signal, signal_type should be selected as 'ECG'. 

In [ ]:
#Filter ECG signal by using predefined filters
filtered_ecg=biobss.preprocess.filter_signal(sig, sampling_rate=fs, signal_type='ECG', method='pantompkins')

Next, ECG R peaks should be detected to calculate peak to peak intervals (ppi, rri). 

BIOBSS provides a specialized peak detection function for ECG signal. The function uses 'ecgdetectors' package to implement methods ('pantompkins', 'hamilton' and 'elgendi') and returns a dictionary of R-peak locations and amplitudes.

In [ ]:
#Detect R-peaks of ECG signal using 'pantompkins' method.
locs_peaks=biobss.ecgtools.ecg_peaks(filtered_ecg,fs,'pantompkins')
peaks = filtered_ecg[locs_peaks]

#### __Calculation of HRV Parameters__
<a id="ecg_par"></a>

BIOBSS provides ___get_hrv_features___ function to calculate HRV parameters from ECG signal. When the signal type is selected as 'ECG', the function can take 'ppi' or 'peaks' as argument. If 'ppi' is selected as input_type, ppi intervals must be provided in milliseconds. If 'peaks' is selected as input_type, ppi is calculated by the function itself using peak locations and sampling rate.

In [ ]:
#Calculate HRV parameters using peak to peak intervals
rri = 1000*np.diff(locs_peaks)/fs
ecg_hrv = biobss.hrvtools.get_hrv_features(sampling_rate=fs, signal_type='ECG', input_type='ppi',ppi=rri)

In [ ]:
#Calculate HRV parameters using peak locations
ecg_hrv = biobss.hrvtools.get_hrv_features(sampling_rate=fs, signal_type='ECG', input_type='peaks',peaks_locs=locs_peaks)


The functions ___hrv_time_features___, ___hrv_freq_features___, ___hrv_nl_features___ can be used seperately in order to calculate hrv parameters for a specific domain. Note that, in order to use these functions peak to peak intervals should be provided as input in milliseconds.

In [ ]:
#Calculate time-domain HRV parameters 
ecg_hrv_time = biobss.hrvtools.hrv_time_features(ppi=ppi, sampling_rate=fs)

In [ ]:
#Calculate frequency-domain HRV parameters 
ecg_hrv_freq = biobss.hrvtools.hrv_freq_features(ppi=ppi, sampling_rate=fs)

In [ ]:
#Calculate nonlinear HRV parameters 
ecg_hrv_nl = biobss.hrvtools.hrv_nl_features(ppi=ppi, sampling_rate=fs)